<a href="https://colab.research.google.com/github/gustavoalcantara/data-portfolio/blob/main/pharmacies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**What is the best location in the city to set up a pharmacy?**

Using Bigquery, it is possible to perform spatial analysis. Therefore, the present code demonstrates the creation of a pharmacy attractiveness index by census tract for the whole of Brazil. The data used were CNPJ, CEP and IBGE census tracts. You can check this analysis at this [link](https://www.youtube.com/watch?v=WQruVEizTlc&t=1s).

In [ ]:
#Install the packages
!pip install basedosdados -q

In [ ]:
#Importando as bibliotecas
import basedosdados as bd
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as pc
from plotly.subplots import make_subplots
from urllib.request import urlopen
import json

In [ ]:
query = '''
WITH pop65 AS (
  SELECT
    id_setor_censitario,
    v099 + v100 + v101 + v102 + v103 + v104 + v105 + v106 + v107 + v108 + v109 +
    v110 + v111 + v112 + v113 + v114 + v115 + v116 + v117 + v118 + v119 + v120 +
    v121 + v122 + v123 + v124 + v125 + v126 + v127 + v128 + v129 + v130 +
    v131 + v132 + v133 + v134 AS mais65
  FROM basedosdados.br_ibge_censo_demografico.setor_censitario_idade_total_2010
),
setor AS (
  SELECT
    pop65.id_setor_censitario,
    mais65,
    geometria
  FROM pop65
  INNER JOIN basedosdados.br_geobr_mapas.setor_censitario_2010 c
  ON pop65.id_setor_censitario = c.id_setor_censitario
),
cnpj AS (
  SELECT
    cnpj_basico,
    nome_fantasia,
    centroide, -- Use o centroide da base de CEP
  FROM basedosdados.br_me_cnpj.estabelecimentos a
  INNER JOIN  basedosdados.br_bd_diretorios_brasil.cep b
  ON a.cep = b.cep
  WHERE cnae_fiscal_principal = '4771701'
    AND situacao_cadastral ='2'
	AND data = '2023-09-15'
  AND a.sigla_uf = 'ES'
    ),
setor_farmacias AS (
  SELECT
    setor.id_setor_censitario,
    setor.mais65,
    COUNT(cnpj.cnpj_basico) as num_farmacias
  FROM setor
  LEFT JOIN cnpj
  ON ST_CONTAINS(setor.geometria, cnpj.centroide) -- Use ST_CONTAINS com o centroide da farmácia
  GROUP BY setor.id_setor_censitario, setor.mais65
),
atratividade AS (
  SELECT
    setor_farmacias.id_setor_censitario,
    setor_farmacias.mais65,
    setor_farmacias.num_farmacias,
    setor.geometria,
    CASE
      WHEN setor_farmacias.num_farmacias = 0 THEN NULL
      ELSE setor.mais65 / NULLIF(setor_farmacias.num_farmacias, 0)
    END as atratividade
  FROM setor_farmacias
  JOIN setor
  ON setor_farmacias.id_setor_censitario = setor.id_setor_censitario
)
SELECT * FROM atratividade
WHERE num_farmacias > 0
'''

df = bd.read_sql(query, billing_project_id='basedosdados-dev')

Downloading: 100%|██████████|


In [ ]:
df.head()

,id_setor_censitario,mais65,num_farmacias,geometria,atratividade
0,320120905000006,42,2,"POLYGON((-41.1103003826434 -20.8523308689393, ...",21.000000
1,320530910110154,67,1,"POLYGON((-40.2924910208275 -20.2800752033543, ...",67.000000
2,320115905000014,35,11,"POLYGON((-41.2906108848764 -20.1528333451184, ...",3.181818
3,320320505000004,77,1,"POLYGON((-40.0696989259583 -19.4022994042483, ...",77.000000
4,320150605000155,75,4,"POLYGON((-40.6183931841516 -19.5220835336526, ...",18.750000
